# Note

Matrix Factorization

# Import

In [2]:
import io
import os
import math
import copy
import pickle
import zipfile
from textwrap import wrap
from pathlib import Path
from itertools import zip_longest
from collections import defaultdict
from urllib.error import URLError
from urllib.request import urlopen

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F 
from torch.optim.lr_scheduler import _LRScheduler

In [2]:
# for Tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/mf')

In [3]:
plt.style.use('ggplot')

In [4]:
def set_random_seed(state=1):
    gens = (np.random.seed, torch.manual_seed, torch.cuda.manual_seed)
    for set_state in gens:
        set_state(state)

In [5]:
RANDOM_STATE = 1
set_random_seed(RANDOM_STATE)

# Download Data

In [6]:
def try_download(url, download_path):
    archive_name = url.split('/')[-1]
    folder_name, _ = os.path.splitext(archive_name)
    
    try:
        r = urlopen(url)
    except URLError as e:
        print('Cannot download the data. Error: %s' % s)
        return 

    assert r.status == 200
    data = r.read()

    with zipfile.ZipFile(io.BytesIO(data)) as arch:
        arch.extractall(download_path)
        
    print('The archive is extracted into folder: %s' % download_path)

In [7]:
archive_url = f'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
download_path =  Path().absolute() / 'data' / 'movielens'
try_download(archive_url, download_path)


The archive is extracted into folder: /Users/tien/Documents/PythonEnvs/pytorch/jup/recsys_models/matrix_fractorization/data/movielens


# Read data

In [8]:
def read_data(path):
    files = {}
    for filename in path.glob('*'):
        if filename.suffix == '.csv':
            files[filename.stem] = pd.read_csv(filename)
        elif filename.suffix == '.dat':
            if filename.stem == 'ratings':
                columns = ['userId', 'movieId', 'rating', 'timestamp']
            else:
                columns = ['movieId', 'title', 'genres']
            data = pd.read_csv(filename, sep='::', names=columns, engine='python', encoding='latin-1')
            files[filename.stem] = data
    return files['ratings'], files['movies']

In [9]:
ratings, movies = read_data(download_path /'ml-1m')

# Dataset Preview

In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## Tabular Preview

In [12]:
def tabular_preview(ratings, n=15):
    """Creates a cross-tabular view of users vs movies."""
    
    user_groups = ratings.groupby('userId')['rating'].count()
    top_users = user_groups.sort_values(ascending=False)[:15]

    movie_groups = ratings.groupby('movieId')['rating'].count()
    top_movies = movie_groups.sort_values(ascending=False)[:15]

    top = (
        ratings.
        join(top_users, rsuffix='_r', how='inner', on='userId').
        join(top_movies, rsuffix='_r', how='inner', on='movieId'))

    return pd.crosstab(top.userId, top.movieId, top.rating, aggfunc=np.sum)

In [13]:
tabular_preview(ratings, movies)

movieId,110,260,480,589,593,608,1196,1198,1210,1270,1580,2028,2571,2762,2858
userId,,,,,,,,,,,,,,,
889,4.0,4.0,3.0,5.0,5.0,4.0,4.0,NaN,3.0,4.0,3.0,3.0,5.0,NaN,2.0
1015,4.0,5.0,4.0,5.0,5.0,5.0,4.0,5.0,4.0,4.0,4.0,5.0,5.0,5.0,4.0
1150,2.0,5.0,NaN,2.0,3.0,5.0,4.0,2.0,3.0,2.0,2.0,2.0,1.0,2.0,4.0
1181,3.0,4.0,2.0,5.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,4.0,5.0,4.0,3.0
1449,3.0,3.0,2.0,2.0,5.0,5.0,3.0,4.0,2.0,2.0,4.0,3.0,4.0,4.0,4.0
1680,1.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,4.0,5.0,3.0,5.0,5.0
1941,5.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0,1.0
1980,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0
2063,5.0,4.0,4.0,2.0,5.0,2.0,4.0,4.0,4.0,4.0,3.0,2.0,5.0,4.0,5.0


# Dataset Preparation

In [14]:
def create_dataset(ratings, top=None):
    """
        Not entirely sure what this does. The returned dataframes are just
        the splitted of the ratings.
    """
    if top is not None:
        ratings.groupby('userId')['rating'].count()
    
    unique_users = ratings.userId.unique()
    user_to_index = {user_id: index for index, user_id in enumerate(unique_users)}
    new_users = ratings.userId.map(user_to_index)
    
    unique_movies = ratings.movieId.unique()
    movie_to_index = {movie_id: index for index, movie_id in enumerate(unique_movies)}
    new_movies = ratings.movieId.map(movie_to_index)
    
    n_users = unique_users.shape[0]
    n_movies = unique_movies.shape[0]
    
    X = pd.DataFrame({'user_id': new_users, 'movie_id': new_movies})
    y = ratings['rating'].astype(np.float32)
    return (n_users, n_movies), (X, y), (user_to_index, movie_to_index)

In [15]:
(n, m), (X, y), _ = create_dataset(ratings)
print(f'Embeddings: {n} users, {m} movies')
print(f'Dataset shape: {X.shape}')
print(f'Target shape: {y.shape}')

Embeddings: 6040 users, 3706 movies
Dataset shape: (1000209, 2)
Target shape: (1000209,)


# Iterator

In [16]:
class RatingsIterator:
    
    def __init__(self, user_movie_matrix, ratings, batch_size=32, shuffle=True):
        
        user_movie_matrix, ratings = np.asarray(user_movie_matrix), np.asarray(ratings)
        
        if shuffle:
            # X.shape[0] is an interger, so we just want to return a list of indices here
            index = np.random.permutation(user_movie_matrix.shape[0])
            
            user_movie_matrix = user_movie_matrix[index]
            ratings = ratings[index]
            
        self.user_movie_matrix = user_movie_matrix
        self.ratings = ratings
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_batches = int(math.ceil(user_movie_matrix.shape[0] // batch_size))
        self._current = 0
        
    def __iter__(self):
        return self
        
    def __next__(self):
        return self.next()
    
    def next(self):
        
        if self._current >= self.n_batches:
            raise StopIteration()
    
        k = self._current
        self._current += 1
        bs = self.batch_size
        
        start_index, end_index = k*bs, (k+1)*bs
        
        return self.user_movie_matrix[start_index:end_index], self.ratings[start_index:end_index]

In [37]:
def batches(user_rating_matrix, ratings, batch_size=32, shuffle=True):
    for xb, yb in RatingsIterator(user_movie_matrix=user_rating_matrix, ratings=ratings, batch_size=batch_size, shuffle=shuffle):
        xb = torch.LongTensor(xb)
        yb = torch.FloatTensor(yb)
        
        '''
        FOr matrix factorization, we have the issue below.
        Thus, we wont need to make a change in shape
        /Users/tien/Documents/PythonEnvs/pytorch/env/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2000, 1])) that is different to the input size (torch.Size([2000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
        return F.mse_loss(input, target, reduction=self.reduction)
        
        '''
        yield xb, yb

# Build Model

In [34]:
# nn.Module is the base class for all the network

class MF(nn.Module):
    """
    Creates a dense network with embedding layers.
    
    Args:
    
        n_users:            
            Number of unique users in the dataset.

        n_movies: 
            Number of unique movies in the dataset.

        n_factors: 
            Number of columns in the embeddings matrix.     
    """
    
    def __init__(self, n_users, n_movies, n_factors=50):
        
        super().__init__() # because we subclass nn.Module
        
        # Build the matrix to store the embedding for n_users
        # Embedding: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
        # A simple look up table that stores embeddingof a fixed dictionary and size.
        # n_users: n number of embeddings
        self.u = nn.Embedding(n_users, n_factors)
        
        # Build the matrix to store the embedding for m_movies
        # A  look up table that stores embeddings for n_movies
        self.m = nn.Embedding(n_movies, n_factors)
        
        # Initialize the embedding vector
        nn.init.normal_(self.u.weight, 0, 0.1)
        nn.init.normal_(self.m.weight, 0, 0.1)
        

        
    def forward(self, users, movies): 
        users_latent = self.u(users)
        movies_latent = self.m(movies)
        
        # Need to understand what dim = 1 here means (TODO)
        # Need to understand what this does here
        # Note:
        # The passed in information is the pair (user, movie).
        # Meaning: 
        #   users is a list of users
        #   movies is a list of movies
        #   The two lists have the same size
        #   At a given index i, we compute the rating 
        #   of a user @ users[i] with regards to a movie @ movies[i]
        # we do not do a matrix multiplication here as this is pair-wise computation
        # For more understanding, see the note here:
        # https://developers.google.com/machine-learning/recommendation/collaborative/matrix
        # Note: Observe that the (i,j) entry of U . V (transpose) of the embeddings of
        # a user i and item j
        # 
        # Below is the dot product of the two embeddinds with the same len
        # we sum() so that each user & movie pair only have one value (ratin)
        return (users_latent * movies_latent).sum(dim=1) 

In [30]:
MF(n, m, n_factors=50)

MF(
  (u): Embedding(6040, 50)
  (m): Embedding(3706, 50)
)

# Build Cyclical Learning Rate

**Cyclical Learning Rate (CLR)**

One of the fastai library features is the cyclical learning rate scheduler. We can implement something similar inheriting the _LRScheduler class from the torch library. Following the original paper's pseudocode, this CLR Keras callback implementation, and making a couple of adjustments to support cosine annealing with restarts, let's create our own CLR scheduler.

The implementation of this idea is quite simple. The base PyTorch scheduler class has the get_lr() method that is invoked each time when we call the step() method. The method should return a list of learning rates depending on the current training epoch. In our case, we have the same learning rate for all of the layers, and therefore, we return a list with a single value.

The next cell defines a CyclicLR class that expectes a single callback function. This function should accept the current training epoch and the base value of learning rate, and return a new learning rate value

In [24]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        # base_lrs: https://github.com/pytorch/pytorch/blob/master/torch/optim/lr_scheduler.py
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [25]:
# Not really sure what this function does so I just put it here
def triangular(step_size, max_lr, method='triangular', gamma=0.99):
    
    def scheduler(epoch, base_lr):
        period = 2 * step_size
        cycle = math.floor(1 + epoch/period)
        x = abs(epoch/step_size - 2*cycle + 1)
        delta = (max_lr - base_lr)*max(0, (1 - x))

        if method == 'triangular':
            pass  # we've already done
        elif method == 'triangular2':
            delta /= float(2 ** (cycle - 1))
        elif method == 'exp_range':
            delta *= (gamma**epoch)
        else:
            raise ValueError('unexpected method: %s' % method)
            
        return base_lr + delta
        
    return scheduler

In [26]:
# This needs to note. Dont know why
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + math.cos(math.pi*t/t_max))/2
    
    return scheduler

In [27]:
def plot_lr(schedule):
    ts = list(range(1000))
    y = [schedule(t, 0.001) for t in ts]
    plt.plot(ts, y)

# Training

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
datasets = {'train': (X_train, y_train), 'val': (X_valid, y_valid)}
dataset_sizes = {'train': len(X_train), 'val': len(X_valid)}

In [21]:
X_train

,user_id,movie_id
529184,3270,1925
341591,2011,579
470922,2898,1497
630004,3807,737
131938,853,536
...,...,...
491263,3019,1158
791624,4731,560
470924,2898,964
491755,3024,411


In [22]:
type(X_train)

pandas.core.frame.DataFrame

In [35]:
model = MF(
    n_users=n,
    n_movies=m,
    n_factors=50
)

## Training Model

In [40]:
lr = 1e-3
wd = 1e-5  # weight decay (TODO)
bs = 2000  # batch size
n_epochs = 10
patience = 10
no_improvements = 0
best_loss = np.inf
best_weights = None
history = []
lr_history = []

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model.to(device)
criterion = nn.MSELoss(reduction='sum')  # criterion
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)  # TODO
iterations_per_epoch = int(math.ceil(dataset_sizes['train'] // bs))
scheduler = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/10))

# Maybe train just on one epoch
add_model_to_tensorboard = False

for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase in ('train', 'val'):
        if phase == 'train':
          training = True
        else:
          training = False

        running_loss = 0
        n_batches = 0
        
        for batch in batches(*datasets[phase], shuffle=training, batch_size=bs):
            x_batch, y_batch = [b.to(device) for b in batch]
            
            # learn up the gradient after each batch
            optimizer.zero_grad()

          
            if not add_model_to_tensorboard:
                writer.add_graph(model, [x_batch[:,0], x_batch[:,1]])
                # writer.close()
                add_model_to_tensorboard = True 
                
            # compute gradients only during 'train' phase
            with torch.set_grad_enabled(training):
                # x_batch[:,0]: take everything in the first (0-index row): users
                # y_batch[:,1]: take everything in the first (1-index row): movies
                
                # call the forward method
                # only take in the available user movie pairs here
                # not a whole user x movie pair
                # tha can be explain why 
                outputs = model(x_batch[:,0], x_batch[:,1])
                preds = torch.round(outputs)  # look into this
                loss = criterion(outputs, y_batch)
                
                # don't update weights and rates when in 'val' phase
                if training:

                    # loss.backward() computes dloss/dx for every parameter 
                    # x which has requires_grad=True. 
                    # These are accumulated into x.grad for every parameter x. In pseudo-code:
                    # x.grad += dloss/dx
                    # source: https://discuss.pytorch.org/t/what-does-the-backward-function-do/9944
                    loss.backward()
                    
                    # optimizer.step updates the value of x using the gradient x.grad. 
                    # For example, the SGD optimizer performs:
                    # x += -lr * x.grad
                    # optimizer.zero_grad() -> should not put it here as the eval value is really bad
                    optimizer.step()
                    
                    # we call it so that the learning rate will change after each epoch
                    # https://discuss.pytorch.org/t/what-does-scheduler-step-do/47764
                    scheduler.step()
                    
                    # relationship between loss.backward() and optimizer.step()
                    # https://stackoverflow.com/questions/53975717/pytorch-connection-between-loss-backward-and-optimizer-step
                    
                    # This is just to save the lr rate to lr_history
                    # so that we can plot later.
                    lr_history.extend(scheduler.get_lr())
                    
            running_loss += loss.item()
            
            # If we have a longer epoch, then we can just add the loss every 100 epoch or so
            # In this case, the number of epoch is less, so we can just add that in here.
            writer.add_scalar('training_loss', running_loss, global_step=epoch)
            
        epoch_loss = running_loss / dataset_sizes[phase]
        stats[phase] = epoch_loss
        
        # early stopping: save weights of the best model so far
        if phase == 'val':
            if epoch_loss < best_loss:
                print('loss improvement on epoch: %d' % (epoch + 1))
                best_loss = epoch_loss
                best_weights = copy.deepcopy(model.state_dict())
                no_improvements = 0
            else:
                no_improvements += 1
                
    history.append(stats)
    print('[{epoch:03d}/{total:03d}] train: {train:.4f} - val: {val:.4f}'.format(**stats))
    if no_improvements >= patience:
        print('early stopping after epoch {epoch:03d}'.format(**stats))
        break
    
writer.flush()


loss improvement on epoch: 1
[001/010] train: 0.6556 - val: 0.7519
loss improvement on epoch: 2
[002/010] train: 0.6333 - val: 0.7487
[003/010] train: 0.6357 - val: 0.7487
loss improvement on epoch: 4
[004/010] train: 0.6157 - val: 0.7461
[005/010] train: 0.6183 - val: 0.7473
loss improvement on epoch: 6
[006/010] train: 0.5981 - val: 0.7449
[007/010] train: 0.6007 - val: 0.7468
[008/010] train: 0.5805 - val: 0.7450
[009/010] train: 0.5832 - val: 0.7476
[010/010] train: 0.5628 - val: 0.7461


In [ ]:
model_path =  Path().absolute() / 'model' 
model_path = f"{model_path}/model"
torch.save(model.state_dict(), model_path)

# Draft

In [4]:
user_em = nn.Embedding(4,5)
movie_em = nn.Embedding(3,5)

nn.init.normal_(user_em.weight, 0, 0.1)
nn.init.normal_(movie_em.weight, 0, 0.1)

Parameter containing:
tensor([[ 0.0456, -0.0329, -0.1824,  0.0988,  0.0981],
        [ 0.0313,  0.1045, -0.1138,  0.0350,  0.1663],
        [ 0.0442, -0.0446, -0.1363,  0.1165,  0.1241]], requires_grad=True)

In [6]:
user_em.weight

Parameter containing:
tensor([[-0.0853,  0.0190,  0.1144,  0.1443,  0.2008],
        [-0.0813, -0.0389,  0.0381, -0.0106,  0.0027],
        [-0.0173,  0.0391, -0.0482,  0.0240, -0.0487],
        [ 0.0896, -0.0158,  0.0979, -0.0678, -0.0472]], requires_grad=True)

In [7]:
movie_em.weight

Parameter containing:
tensor([[ 0.0456, -0.0329, -0.1824,  0.0988,  0.0981],
        [ 0.0313,  0.1045, -0.1138,  0.0350,  0.1663],
        [ 0.0442, -0.0446, -0.1363,  0.1165,  0.1241]], requires_grad=True)

In [10]:
test = user_em * movie_em

TypeError: unsupported operand type(s) for *: 'Embedding' and 'Embedding'

In [11]:
# Need to understanding this better
users_latent = user_em(torch.IntTensor([0,1]))

In [12]:
users_latent # 2 x 5

tensor([[-0.0853,  0.0190,  0.1144,  0.1443,  0.2008],
        [-0.0813, -0.0389,  0.0381, -0.0106,  0.0027]],
       grad_fn=<EmbeddingBackward0>)

In [41]:
movies_latent = movie_em(torch.IntTensor([0,1,2]))

In [42]:
movies_latent # 2 x 5

tensor([[ 0.0456, -0.0329, -0.1824,  0.0988,  0.0981],
        [ 0.0313,  0.1045, -0.1138,  0.0350,  0.1663],
        [ 0.0442, -0.0446, -0.1363,  0.1165,  0.1241]],
       grad_fn=<EmbeddingBackward0>)

In [43]:
users_latent * movies_latent

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 0

In [17]:
-0.0190 * -0.0391

0.0007429

In [18]:
(users_latent * movies_latent).sum(dim=1)

tensor([-0.0096, -0.0023], grad_fn=<SumBackward1>)

In [20]:
(users_latent * movies_latent).sum(dim=-1)

tensor([-0.0096, -0.0023], grad_fn=<SumBackward1>)

In [19]:
sum([ 0.0015,  0.0007, -0.0055,  0.0035, -0.0098])

-0.0096

In [3]:
download_path =  Path().absolute() / 'data' / 'movielens'

In [2]:
from pathlib import Path

In [4]:
type(download_path)

pathlib.PosixPath